In [23]:
import pandas as pd
import lzma
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem

In [24]:
# Read the data to a pandas dataframe
ach = pd.read_csv(
    "../../data/Acetylcholinesterase_human_IC50_ChEMBLBindingDB_spliton6_binary_1micromolar.csv"
)
ach.head()

,SMILES,InChI,chem-id,chem-name,single-class-label
0,CN(C)C(=O)Oc1cccc([N+](C)(C)C)c1,InChI=1S/C12H19N2O2/c1-13(2)12(15)16-11-8-6-7-...,CHEMBL54126,NEOSTIGMINE BROMIDE,1
1,O=C(CCCCC1CCSS1)NCCCNc1c2c(nc3cc(Cl)ccc13)CCCC2,InChI=1S/C24H32ClN3OS2/c25-17-10-11-20-22(16-1...,CHEMBL194823,LIPOCRINE,1
2,COc1cc2c(cc1OC)C(=O)C(CC1CCN(CCCNc3c4c(nc5cc(C...,InChI=1S/C33H40ClN3O3/c1-39-30-18-22-17-23(33(...,CHEMBL3216655,NaN,1
3,CNC(=O)Oc1cccc(CN(C)CCCOc2ccc3c(=O)c4ccccc4oc3...,InChI=1S/C26H26N2O5/c1-27-26(30)32-20-8-5-7-18...,CHEMBL340427,XANTHOSTIGMINE,1
4,CCC1=CC2Cc3nc4cc(Cl)ccc4c(N)c3[C@@H](C1)C2,InChI=1S/C18H19ClN2/c1-2-10-5-11-7-12(6-10)17-...,CHEMBL208599,HUPRINE X,1


In [25]:
fplen = 1024


def smiles2ecfp(smi):
    fpgen = AllChem.GetMorganGenerator(radius=3, fpSize=fplen)
    m1 = Chem.MolFromSmiles(smi)
    return fpgen.GetFingerprint(m1)


def set_bit_percent(fp):
    s = fp.ToBitString()
    return s.count("1") / len(s)


def bitstring2bytes(s):
    return int(s, 2).to_bytes((len(s) + 7) // 8, byteorder="big")


ach["ecfp"] = ach.apply(lambda row: smiles2ecfp(row["SMILES"]), axis=1)
ach["set-bit-percent"] = ach.apply(lambda row: set_bit_percent(row["ecfp"]), axis=1)
print("Average % of ones: " + str(ach["set-bit-percent"].mean()))
print("Average count of ones: " + str(ach["set-bit-percent"].mean() * fplen))

Average % of ones: 0.06716784892638036
Average count of ones: 68.77987730061349


In [26]:
test = ach["ecfp"][0].ToBitString()
print(test)
print(len(test))

0000000000000001000000001000000001100000000000000000000000000000100000000000000000000000001000000000000000000000000010000000001000000000000100000000000100000000000000000001000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000100000000000000010000000000000000000000000000000000000000000001000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000001000100100000000001000000000000000000000000000001000000000000000000000000110000000000000000000011000000000000000000000000000000000000000000010000000001000000000000000000001000000000000000010000000010000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000001000000000000000000000000010000010000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [27]:
# Creating tree nodes
class NodeTree(object):
    def __init__(self, left=None, right=None):
        self.left = left
        self.right = right

    def children(self):
        return self.left, self.right

    def nodes(self):
        return self.left, self.right

    def __str__(self):
        return f"{self.left}_{self.right}"


# Main function implementing huffman coding
def huffman_code_tree(node, left=True, binString=""):
    if type(node) is int:
        return {node: binString}
    (l, r) = node.children()
    d = dict()
    d.update(huffman_code_tree(l, True, binString + "0"))
    d.update(huffman_code_tree(r, False, binString + "1"))
    return d


def huffman(by):
    freq = {}
    for b in by:
        if b in freq:
            freq[b] += 1
        else:
            freq[b] = 1

    freq = sorted(freq.items(), key=lambda x: x[1], reverse=True)
    nodes = freq
    # print(freq)
    while len(nodes) > 1:
        (key1, c1) = nodes[-1]
        (key2, c2) = nodes[-2]
        nodes = nodes[:-2]
        node = NodeTree(key1, key2)
        nodes.append((node, c1 + c2))

        nodes = sorted(nodes, key=lambda x: x[1], reverse=True)

    return huffman_code_tree(nodes[0][0])

In [28]:
# repeatedly huffman compress until the compressed string is less than 39 bits
def rhuffman(comp):
    while len(comp) > 39:
        b = bitstring2bytes(comp)
        h = huffman(b)
        comp = "".join([h[c] for c in b])
    print(comp)
    return comp
ach["huff_comp"] = ach.apply(lambda row: rhuffman(row["ecfp"].ToBitString()), axis=1)
# write ach to a file
ach.to_csv("../../data/plasmodium_malaria_subsampled_encoded_train_huffman.csv")

0010000110101101110011111110101100
00011010101100111110
11011111110001000011010101100
1011001111100100
101100111110001000011010
101100111110001000011010
100101110100010000110101101110011111110
100101110100010000110101101110011111110
101100111110001000011010
00011010101100111110
101100111110001000011010
00011010101100111110
101100111110001000011010
00011010101100111110
1011001111100100
00011010101100111110
1011001111100100
00011010101100111110
100101110100010000110101101110011111110
1011001111100100
0010000110101101110011111110101100
1111001011101000100001111101011011100
100101110100010000110101101110011111110
1111001011101000100001111101011011100
101100111110001000011010
1011001111100100
101100111110001000011010
0010000110101101110011111110101100
11011111110001000011010101100
11011111110001000011010101100
1111001011101000100001111101011011100
0010000110101101110011111110101100
1111001011101000100001111101011011100
11011111110001000011010101100
00011010101100111110
001000011010110111001

In [29]:
# trees = []
# comp = test
# tr = []
# while len(comp) > 39:
#     b = bitstring2bytes(comp)
#     h = huffman(b)
#     comp = "".join([h[c] for c in b])
#     tr.append(h)
# trees.append(tr)
# print(comp)